<a href="https://colab.research.google.com/github/samar-singh/llms/blob/main/deepseek_vl2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL2


Cloning into 'DeepSeek-VL2'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 211 (delta 61), reused 40 (delta 40), pack-reused 129 (from 2)
Receiving objects: 100% (211/211), 31.05 MiB | 23.65 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [5]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers datasets torch torchvision torchaudio
#!pip install -r /content/DeepSeek-VL2/requirements.txt
!pip install /content/DeepSeek-VL2
#!pip install -e /content/DeepSeek-VL2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 63.0 MB/s eta 0:00:00
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninsta

In [3]:
!pip install fsspec==2025.3.0

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.0 which is incompatible.
datasets 3.5.0 requires tqdm>=4.66.3, but you have tqdm 4.64.0 which is incompatible.


In [2]:
!pip install pipdeptree
!pipdeptree --warn conflict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
usage: pipdeptree [-h] [-v] [-w [{silence,suppress,fail}]] [--python PYTHON] [--path PATH] [-p P] [-e P] [-l | -u] [-f] [--encoding E] [-a] [-d D] [-r] [--license] [-j | --json-tree | --mermaid | --graph-output FMT]
pipdeptree: error: argument -w/--warn: invalid choice: 'conflict' (choose from 'silence', 'suppress', 'fail')


In [9]:
import torch
from transformers import AutoModelForCausalLM
import os
import torch, gc

gc.collect()
torch.cuda.empty_cache()

# Add this to your notebook before your generation code
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

# Convert model to FP16 if not done yet

#vl_gpt.half()

# Enable checkpointing
#vl_gpt.gradient_checkpointing_enable()

In [13]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images


# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()
# multiple images/interleaved image-text
conversation = [
    {
        "role": "<|User|>",
        "content": #"This is image_1: <image>\n"
                   #"This is image_2: <image>\n"
                   #"This is image_3: <image>\n Can you tell me what are in the images?",
                   "Respond in Markdown\n"
                   "This is 20250325-1946_page_1 : <image>Extract the text \n"
                   #"This is image_1: <image>\n This is a Question paper .Can you do the following :\n"
                   #"Step-1 Extract the question text \n"
                   #"Step-2 Extract the marks for each question\n"
                   "What is this image ?",
                   #"Step-4 Respond with all the data collected in a table with columns Sl. No , Question , Marks, Answer",
        "images": [
            #"images/multi_image_1.jpeg",
            #"images/multi_image_2.jpeg",
            #"images/multi_image_3.jpeg",
            #"/content/drive/MyDrive/AI_ML/EvaluationProject/output_questions/converted_images/20250325-1946/20250325-1946_page_1.jpg",
            "/content/drive/MyDrive/AI_ML/EvaluationProject/output/converted_images/Answer_sheet_5/Answer_sheet_5_page_2.jpg",
            #"/content/drive/MyDrive/AI_ML/EvaluationProject/output_questions/converted_images/20250325-1946/20250325-1946_page_1.jpg",
        ],
    },
    {"role": "<|Assistant|>", "content": ""}
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False)
print(f"{prepare_inputs['sft_format'][0]}", answer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<|User|>: Respond in Markdown
This is 20250325-1946_page_1 : <image>Extract the text 
What is this image ?

<|Assistant|>: This image is a handwritten note discussing the production of viable seeds for a specific type of plant. It outlines the steps involved in producing 200 pollen grains, which are necessary for fertilization and seed formation. The text includes mathematical calculations and explanations of the process.<｜end▁of▁sentence｜>
